In [78]:
import json
import numpy as np
import pandas as pd
from scipy import stats


In [124]:
cell_list = json.load(open('data/cell_list.json', 'r'))
cell_list = sorted(set(cell_list['cells']) - set(['UNK']))
cell_ctl_exp = pd.read_csv('data/cell_ctl_exp.csv', index_col=0)
cell_embeddings = pd.read_csv('data/cell_embeddings_similarity.csv', index_col=0)

In [125]:
cell_ctl_exp = cell_ctl_exp.loc[cell_list]
cell_ctl_exp 

,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
cell_name,,,,,,,,,,,,,,,,,,,,,
1HAE,-0.058008,-0.081057,-0.422165,0.033539,0.049038,0.081435,0.259071,-0.044254,-0.162816,0.013538,...,-0.006116,0.359219,0.126941,-0.332527,-0.021451,-0.409765,0.150805,-0.107482,-0.148919,0.120752
22RV1,-0.460715,-0.298550,0.276651,0.227987,0.166067,-0.329886,-0.305881,-0.437880,0.388337,-0.393604,...,0.227711,-0.019580,0.695980,-0.140347,-0.218419,-0.013262,0.383915,0.186184,0.119527,-0.082171
5637,-0.221060,-0.265016,-0.203910,-0.313397,-0.637892,1.202088,0.353803,-0.395260,0.096077,-0.293992,...,0.003067,-0.661664,-0.656039,0.306619,0.026369,-0.345291,-0.604470,0.055729,0.168482,-0.230683
A375,0.423214,0.329367,-0.113771,0.509798,0.999382,0.138161,-0.792899,0.445613,0.876968,0.543444,...,-0.178942,0.489548,-0.163790,-1.068656,0.350942,1.028036,-0.463699,0.499797,0.886370,0.348604
A549,1.718592,2.481547,0.133010,2.058260,-0.011534,0.947378,2.773531,3.147931,1.449704,2.524118,...,0.401789,0.313758,-1.718147,0.594781,1.075256,1.699572,-2.761166,-1.899817,1.563429,1.480171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XC.P934,-0.419000,-0.696000,-0.908700,-0.219050,0.582050,-0.555850,-0.399000,-0.337600,0.811750,0.425550,...,0.445900,-0.545300,0.658150,-1.000200,-0.624950,-0.631200,-0.074600,-0.687700,-1.691900,-0.788700
XC.P935,-0.256172,0.204494,-0.935846,-0.441512,-0.160283,0.597977,-0.369535,-0.071572,0.083955,-0.117166,...,-0.336476,0.642213,-0.061178,-1.388283,-0.144158,0.226131,-0.140572,0.163104,-0.079241,0.651406
XC.P936,-0.635150,-0.336850,-0.886750,-0.672650,0.634550,0.074100,-0.042150,-0.656900,-0.261100,0.584950,...,-1.255000,0.204850,0.705250,-0.488300,-1.046600,0.019000,0.342900,-0.264850,-1.054850,-0.647500


In [139]:

def calculate_spearman(data, query):
   # Calculate correlation for each row
   assert query.shape[1] == 978
   correlations = []
   for i in range(query.shape[0]):
      scorr = []
      for k in range(data.shape[0]):
         corr, _ = stats.spearmanr(data.iloc[k], query.iloc[i])
         scorr.append(corr)
      correlations.append(scorr)
   return pd.DataFrame(correlations, index=query.index.tolist(), columns=data.index.tolist())

In [140]:
#query
query = pd.DataFrame(np.ndarray(978).reshape(1,-1), index=['UNK'])
############################
query_correlations = calculate_spearman(cell_ctl_exp, query)

In [142]:
new_cell_list = cell_list + query.index.tolist()
new_cell_embeddings = pd.concat([cell_embeddings, query_correlations], axis=0)
new_cell_embeddings = pd.DataFrame(new_cell_embeddings, index=new_cell_list, columns=cell_list)

In [ ]:
new_cell_embeddings.to_csv('data/cell_embeddings_similarity.csv',)